In [1]:
import os
import FinanceDataReader as fdr
from tqdm import tqdm
import pandas as pd

# Pycaret 이용

## 모든 종목 예측

* 종가와 상관계수 높은 특성 : KOSDAQ(0.87), KOSPI(0.79), DOW(0.58), NASDAQ(0.50), S&P500(0.56)
* 예측 순서(종가와 상관계수 높은 특성 먼저 예측)
    * weekday, weeknumdm로 KOSDAQ 예측(종가와 상관계수 0.87)
    * weekday, weeknum, KOSDAQ으로 NASDAQ 예측(KOSDAQ과 상관계수 0.68)
    * weekday, weeknum, KOSDAQ, NASDAQ으로 S&P500 예측(NASDAQ과 상관계수 0.93)
    * weekday, weeknum, KOSDAQ, NASDAQ, S&P500으로 DOW 예측(S&P500과 상관계수 0.95)
    * weekday, weeknum, KOSDAQ, NASDAQ, S&P500, DOW로 KOSPI 예측
    * weekday, weeknum, KOSDAQ, NASDAQ, S&P500, DOW, KOSPI로 다른 종가 예측 

In [2]:
# 제출 점수 :

In [3]:
from pycaret.regression import *

In [4]:
# 외부 데이터 가져오기
df = pd.read_csv('../data/20210104_20211119_data_interpolate.csv')
df.head()

,Date,exchange_rate,kospi,kosdaq,kospi100,dow,nasdaq,sp500,미국채10년-2년,BTC,Gold,Oil,US10Y,US2Y
0,2021-01-04,1086.48,2944.45,977.62,3052.74,30223.89,12698.4,3700.65,0.8009,36499000.0,37.10,32.38,0.918,0.1171
1,2021-01-05,1086.42,2990.57,985.76,3099.42,30391.60,12819.0,3726.86,0.8320,37962000.0,37.18,33.94,0.955,0.1230
2,2021-01-06,1087.93,2968.21,981.39,3070.05,30829.40,12740.8,3748.14,0.9002,40731000.0,36.60,34.11,1.039,0.1388
3,2021-01-07,1094.28,3031.68,988.86,3139.48,31041.13,13067.5,3803.79,0.9402,44575000.0,36.48,34.53,1.081,0.1408
4,2021-01-08,1092.93,3152.18,987.79,3293.96,31097.97,13202.0,3824.68,0.9821,47241000.0,35.26,35.43,1.119,0.1369


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 230 entries, 0 to 229
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Date           230 non-null    object 
 1   exchange_rate  230 non-null    float64
 2   kospi          230 non-null    float64
 3   kosdaq         230 non-null    float64
 4   kospi100       230 non-null    float64
 5   dow            230 non-null    float64
 6   nasdaq         230 non-null    float64
 7   sp500          230 non-null    float64
 8   미국채10년-2년      230 non-null    float64
 9   BTC            230 non-null    float64
 10  Gold           230 non-null    float64
 11  Oil            230 non-null    float64
 12  US10Y          230 non-null    float64
 13  US2Y           230 non-null    float64
dtypes: float64(13), object(1)
memory usage: 25.3+ KB


In [6]:
df.columns

Index(['Date', 'exchange_rate', 'kospi', 'kosdaq', 'kospi100', 'dow', 'nasdaq',
       'sp500', '미국채10년-2년', 'BTC', 'Gold', 'Oil', 'US10Y', 'US2Y'],
      dtype='object')

# 필요한 특성만 추출

In [7]:
df = df[['Date','kosdaq','nasdaq','sp500','dow','kospi']]
df.head()

,Date,kosdaq,nasdaq,sp500,dow,kospi
0,2021-01-04,977.62,12698.4,3700.65,30223.89,2944.45
1,2021-01-05,985.76,12819.0,3726.86,30391.60,2990.57
2,2021-01-06,981.39,12740.8,3748.14,30829.40,2968.21
3,2021-01-07,988.86,13067.5,3803.79,31041.13,3031.68
4,2021-01-08,987.79,13202.0,3824.68,31097.97,3152.18


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 230 entries, 0 to 229
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Date    230 non-null    object 
 1   kosdaq  230 non-null    float64
 2   nasdaq  230 non-null    float64
 3   sp500   230 non-null    float64
 4   dow     230 non-null    float64
 5   kospi   230 non-null    float64
dtypes: float64(5), object(1)
memory usage: 10.9+ KB


In [9]:
# 종목 정보 가져오기
path = '../data'
list_name = 'Stock_List.csv'
stock_list = pd.read_csv(os.path.join(path,list_name))
stock_list.head()

,종목명,종목코드,상장시장
0,삼성전자,5930,KOSPI
1,SK하이닉스,660,KOSPI
2,NAVER,35420,KOSPI
3,카카오,35720,KOSPI
4,삼성바이오로직스,207940,KOSPI


In [10]:
stock_list.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 370 entries, 0 to 369
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   종목명     370 non-null    object
 1   종목코드    370 non-null    int64 
 2   상장시장    370 non-null    object
dtypes: int64(1), object(2)
memory usage: 8.8+ KB


In [11]:
# 종목 코드 6자리로 맞추기
stock_list['종목코드'] = stock_list["종목코드"].astype(str).str.zfill(6)
stock_list.head()

,종목명,종목코드,상장시장
0,삼성전자,005930,KOSPI
1,SK하이닉스,000660,KOSPI
2,NAVER,035420,KOSPI
3,카카오,035720,KOSPI
4,삼성바이오로직스,207940,KOSPI


# 종가 예측

* 1. kosdaq 예측
* 2. nasdaq
* 3. sp500
* 4. dow
* 5. kospi
* 6. 종가

In [12]:
start_date = '20210104'
end_date = '20211119'
Business_days = pd.DataFrame(pd.date_range(start_date,end_date,freq='B'), columns = ['Date'])

## 기본 + 5개 + 종목 데이터 프레임 만들기
* 2021년 1월 4일 ~ 11월 19일

In [13]:
# 종목 주가 정보 가져오기
code = stock_list['종목코드'][0]
basic_data = fdr.DataReader(code, start = start_date, end = end_date)[['Close']].reset_index()
# 공휴일 포함한 평일 날짜로 데이터 셋 채우기
basic_data = pd.merge(Business_days, basic_data, how = 'outer')
# 요 번호와 주 번호 지정하여 열로 추가하기
basic_data['weekday'] = basic_data.Date.apply(lambda x : x.weekday())
basic_data['weeknum'] = basic_data.Date.apply(lambda x : x.strftime('%V'))
# 빈값 선형으로 채우기
basic_data.Close = basic_data.Close.interpolate()
# data의 Close 열 이름 종목 코드명으로 바꾸기
basic_data = basic_data.rename(columns={'Close':str(code)})
# 날짜 기준으로 외부 데이터와 주가 정보 합치기
basic_data['Date'] = basic_data['Date'].astype('str')
basic_data = pd.merge(df, basic_data, on='Date')
basic_data.head()

,Date,kosdaq,nasdaq,sp500,dow,kospi,005930,weekday,weeknum
0,2021-01-04,977.62,12698.4,3700.65,30223.89,2944.45,83000.0,0,01
1,2021-01-05,985.76,12819.0,3726.86,30391.60,2990.57,83900.0,1,01
2,2021-01-06,981.39,12740.8,3748.14,30829.40,2968.21,82200.0,2,01
3,2021-01-07,988.86,13067.5,3803.79,31041.13,3031.68,82900.0,3,01
4,2021-01-08,987.79,13202.0,3824.68,31097.97,3152.18,88800.0,4,01


In [14]:
basic_data.columns

Index(['Date', 'kosdaq', 'nasdaq', 'sp500', 'dow', 'kospi', '005930',
       'weekday', 'weeknum'],
      dtype='object')

In [15]:
len(basic_data.columns)

9

In [16]:
basic_data = basic_data[['Date', 'weekday', 'weeknum' ,'kosdaq', 'nasdaq', 'sp500', 'dow', 'kospi', '005930']]
basic_data.head()

,Date,weekday,weeknum,kosdaq,nasdaq,sp500,dow,kospi,005930
0,2021-01-04,0,01,977.62,12698.4,3700.65,30223.89,2944.45,83000.0
1,2021-01-05,1,01,985.76,12819.0,3726.86,30391.60,2990.57,83900.0
2,2021-01-06,2,01,981.39,12740.8,3748.14,30829.40,2968.21,82200.0
3,2021-01-07,3,01,988.86,13067.5,3803.79,31041.13,3031.68,82900.0
4,2021-01-08,4,01,987.79,13202.0,3824.68,31097.97,3152.18,88800.0


In [17]:
data = basic_data.copy()

In [18]:
for code in tqdm(stock_list['종목코드'][1:]):
    # 종목 주가 정보 가져오기
    one_stock_data = fdr.DataReader(code, start = start_date, end = end_date)[['Close']].reset_index()
    # 공휴일 포함한 평일 날짜로 데이터 셋 채우기
    one_stock_data = pd.merge(Business_days, one_stock_data, how = 'outer')
    # 빈값 선형으로 채우기
    one_stock_data.Close = one_stock_data.Close.interpolate()
    # data의 Close 열 이름 종목 코드명으로 바꾸기
    one_stock_data = one_stock_data.rename(columns={'Close':str(code)})
    # 날짜 기준으로 외부 데이터와 주가 정보 합치기
    one_stock_data['Date'] = one_stock_data['Date'].astype('str')
    data = pd.merge(data, one_stock_data, on='Date')

100%|██████████| 369/369 [01:16<00:00,  4.83it/s]


In [49]:
data.head()

,Date,weekday,weeknum,kosdaq,nasdaq,sp500,dow,kospi,005930,000660,...,078020,102710,314130,035890,032620,220630,064260,287410,110790,099320
0,2021-01-04,0,01,977.62,12698.4,3700.65,30223.89,2944.45,83000.0,126000.0,...,6740.0,47950.0,84700.0,1610.0,9640.0,3705.0,4835.0,4200.0,25850.0,31950.0
1,2021-01-05,1,01,985.76,12819.0,3726.86,30391.60,2990.57,83900.0,130500.0,...,6780.0,47050.0,82500.0,1700.0,9640.0,3890.0,4820.0,4240.0,24750.0,32950.0
2,2021-01-06,2,01,981.39,12740.8,3748.14,30829.40,2968.21,82200.0,131000.0,...,7000.0,45650.0,81600.0,1705.0,9780.0,3755.0,4795.0,4295.0,24750.0,35350.0
3,2021-01-07,3,01,988.86,13067.5,3803.79,31041.13,3031.68,82900.0,134500.0,...,7150.0,44850.0,78500.0,1725.0,9710.0,3650.0,4810.0,4480.0,24750.0,38200.0
4,2021-01-08,4,01,987.79,13202.0,3824.68,31097.97,3152.18,88800.0,138000.0,...,7140.0,45050.0,77500.0,1705.0,9440.0,3580.0,4755.0,4300.0,23700.0,38300.0


In [50]:
# 총 378열
len(data.columns)

378

## 훈련, 테스트 셋 나누기

In [51]:
train = data[data['Date'] < '2021-11-01']
test =  data[(data['Date'] <= '2021-11-05') & (data['Date'] >= '2021-11-01')]

In [52]:
train.tail()

,Date,weekday,weeknum,kosdaq,nasdaq,sp500,dow,kospi,005930,000660,...,078020,102710,314130,035890,032620,220630,064260,287410,110790,099320
210,2021-10-25,0,43,994.31,15226.7,4566.48,35743.78,3020.54,70200.0,100000.0,...,9070.0,29100.0,36600.0,1865.0,7880.0,4620.0,7690.0,7200.0,43450.0,50200.0
211,2021-10-26,1,43,1011.76,15235.7,4574.79,35755.83,3049.08,71100.0,102000.0,...,9230.0,29950.0,38000.0,1930.0,7960.0,4750.0,7530.0,7180.0,44450.0,49550.0
212,2021-10-27,2,43,1008.95,15235.8,4551.68,35491.48,3025.49,70100.0,101500.0,...,9210.0,29350.0,37450.0,1900.0,7970.0,4840.0,8080.0,7270.0,43750.0,48200.0
213,2021-10-28,3,43,1000.13,15448.1,4596.42,35729.89,3009.55,70700.0,106500.0,...,9140.0,29700.0,38250.0,1860.0,7830.0,4860.0,7620.0,7220.0,42950.0,49750.0
214,2021-10-29,4,43,992.33,15498.4,4605.38,35819.59,2970.68,69800.0,103000.0,...,9050.0,28900.0,37200.0,1840.0,7670.0,4885.0,8220.0,7120.0,42850.0,50200.0


In [53]:
test

,Date,weekday,weeknum,kosdaq,nasdaq,sp500,dow,kospi,005930,000660,...,078020,102710,314130,035890,032620,220630,064260,287410,110790,099320
215,2021-11-01,0,44,998.57,15595.9,4613.67,35913.68,2978.94,69900.0,106500.0,...,9030.0,28950.0,36600.0,1845.0,8050.0,4930.0,8970.0,7480.0,41600.0,50300.0
216,2021-11-02,1,44,1009.44,15649.6,4630.65,36053.09,3013.49,71500.0,107500.0,...,9060.0,29150.0,38000.0,1875.0,8090.0,4965.0,9050.0,7430.0,41650.0,49950.0
217,2021-11-03,2,44,1005.00,15811.6,4660.57,36157.02,2975.71,70400.0,105500.0,...,9040.0,29000.0,38650.0,1830.0,7900.0,4980.0,9080.0,7110.0,42000.0,48850.0
218,2021-11-04,3,44,1001.43,15940.3,4680.06,36124.66,2983.22,70600.0,106000.0,...,9020.0,28750.0,39750.0,1815.0,7870.0,4975.0,8100.0,6990.0,42100.0,47350.0
219,2021-11-05,4,44,1001.35,15971.6,4697.53,36329.07,2969.27,70200.0,107000.0,...,8940.0,28650.0,38400.0,1800.0,7670.0,5010.0,8480.0,6780.0,41850.0,45550.0


# Pycaret으로 각 특성 예측

In [54]:
train.columns[0:8]

Index(['Date', 'weekday', 'weeknum', 'kosdaq', 'nasdaq', 'sp500', 'dow',
       'kospi'],
      dtype='object')

In [55]:
features = list(train.columns)
type(features)

list

In [56]:
end_features = features[:3]
end_features

['Date', 'weekday', 'weeknum']

In [57]:
type(end_features[0])

str

In [58]:
test[end_features]

,Date,weekday,weeknum
215,2021-11-01,0,44
216,2021-11-02,1,44
217,2021-11-03,2,44
218,2021-11-04,3,44
219,2021-11-05,4,44


### 테스트셋의 5가지 특성 예측 하기

In [29]:
for i in range(5):
        model = setup(train, target = features[i+3], ignore_features = features[i+4:], silent=True)
        tuned_model = tune_model(compare_models(sort='MAPE', n_select=1))
        tuned_model_ = finalize_model(tuned_model)
        end_features = features[:i+3]
        pred = predict_model(tuned_model_, test[end_features])
        test[features[i+3]] = pred['Label']

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,15.5284,494.9727,22.2480,0.9390,0.0070,0.0049
1,17.5424,517.4255,22.7470,0.9437,0.0072,0.0056
2,34.2119,1662.4879,40.7736,0.7533,0.0133,0.0112
3,17.2891,536.9035,23.1712,0.9454,0.0073,0.0055
4,33.7926,1974.2821,44.4329,0.7401,0.0142,0.0108
5,21.0461,553.2620,23.5215,0.8517,0.0075,0.0067
6,23.2963,1392.5711,37.3172,0.9088,0.0123,0.0076
7,21.6241,795.9822,28.2132,0.9173,0.0091,0.0070
8,26.9286,1234.6640,35.1378,0.7486,0.0112,0.0085
9,18.0686,681.8829,26.1129,0.9100,0.0084,0.0058


In [59]:
test

,Date,weekday,weeknum,kosdaq,nasdaq,sp500,dow,kospi,005930,000660,...,078020,102710,314130,035890,032620,220630,064260,287410,110790,099320
215,2021-11-01,0,44,998.57,15595.9,4613.67,35913.68,2978.94,69900.0,106500.0,...,9030.0,28950.0,36600.0,1845.0,8050.0,4930.0,8970.0,7480.0,41600.0,50300.0
216,2021-11-02,1,44,1009.44,15649.6,4630.65,36053.09,3013.49,71500.0,107500.0,...,9060.0,29150.0,38000.0,1875.0,8090.0,4965.0,9050.0,7430.0,41650.0,49950.0
217,2021-11-03,2,44,1005.00,15811.6,4660.57,36157.02,2975.71,70400.0,105500.0,...,9040.0,29000.0,38650.0,1830.0,7900.0,4980.0,9080.0,7110.0,42000.0,48850.0
218,2021-11-04,3,44,1001.43,15940.3,4680.06,36124.66,2983.22,70600.0,106000.0,...,9020.0,28750.0,39750.0,1815.0,7870.0,4975.0,8100.0,6990.0,42100.0,47350.0
219,2021-11-05,4,44,1001.35,15971.6,4697.53,36329.07,2969.27,70200.0,107000.0,...,8940.0,28650.0,38400.0,1800.0,7670.0,5010.0,8480.0,6780.0,41850.0,45550.0


In [60]:
train.tail()

,Date,weekday,weeknum,kosdaq,nasdaq,sp500,dow,kospi,005930,000660,...,078020,102710,314130,035890,032620,220630,064260,287410,110790,099320
210,2021-10-25,0,43,994.31,15226.7,4566.48,35743.78,3020.54,70200.0,100000.0,...,9070.0,29100.0,36600.0,1865.0,7880.0,4620.0,7690.0,7200.0,43450.0,50200.0
211,2021-10-26,1,43,1011.76,15235.7,4574.79,35755.83,3049.08,71100.0,102000.0,...,9230.0,29950.0,38000.0,1930.0,7960.0,4750.0,7530.0,7180.0,44450.0,49550.0
212,2021-10-27,2,43,1008.95,15235.8,4551.68,35491.48,3025.49,70100.0,101500.0,...,9210.0,29350.0,37450.0,1900.0,7970.0,4840.0,8080.0,7270.0,43750.0,48200.0
213,2021-10-28,3,43,1000.13,15448.1,4596.42,35729.89,3009.55,70700.0,106500.0,...,9140.0,29700.0,38250.0,1860.0,7830.0,4860.0,7620.0,7220.0,42950.0,49750.0
214,2021-10-29,4,43,992.33,15498.4,4605.38,35819.59,2970.68,69800.0,103000.0,...,9050.0,28900.0,37200.0,1840.0,7670.0,4885.0,8220.0,7120.0,42850.0,50200.0


In [61]:
target_feature = features[9]
basic_features = features[:8] + [target_feature]
basic_features

['Date',
 'weekday',
 'weeknum',
 'kosdaq',
 'nasdaq',
 'sp500',
 'dow',
 'kospi',
 '000660']

In [62]:
ignore_features = features
for col in basic_features:
    ignore_features.remove(col)
len(ignore_features)

369

In [63]:
type(ignore_features)

list

In [64]:
test

,Date,weekday,weeknum,kosdaq,nasdaq,sp500,dow,kospi,005930,000660,...,078020,102710,314130,035890,032620,220630,064260,287410,110790,099320
215,2021-11-01,0,44,998.57,15595.9,4613.67,35913.68,2978.94,69900.0,106500.0,...,9030.0,28950.0,36600.0,1845.0,8050.0,4930.0,8970.0,7480.0,41600.0,50300.0
216,2021-11-02,1,44,1009.44,15649.6,4630.65,36053.09,3013.49,71500.0,107500.0,...,9060.0,29150.0,38000.0,1875.0,8090.0,4965.0,9050.0,7430.0,41650.0,49950.0
217,2021-11-03,2,44,1005.00,15811.6,4660.57,36157.02,2975.71,70400.0,105500.0,...,9040.0,29000.0,38650.0,1830.0,7900.0,4980.0,9080.0,7110.0,42000.0,48850.0
218,2021-11-04,3,44,1001.43,15940.3,4680.06,36124.66,2983.22,70600.0,106000.0,...,9020.0,28750.0,39750.0,1815.0,7870.0,4975.0,8100.0,6990.0,42100.0,47350.0
219,2021-11-05,4,44,1001.35,15971.6,4697.53,36329.07,2969.27,70200.0,107000.0,...,8940.0,28650.0,38400.0,1800.0,7670.0,5010.0,8480.0,6780.0,41850.0,45550.0


In [65]:
test.columns[150:200]

Index(['007310', '051600', '010120', '069620', '069960', '042700', '004000',
       '102780', '336370', '111770', '006650', '000240', '010780', '016380',
       '000150', '009900', '013890', '005850', '001740', '139130', '181710',
       '175330', '005250', '185750', '003000', '051905', '330590', '192820',
       '214980', '122630', '031430', '273130', '032350', '066575', '363280',
       '298000', '278530', '000670', '292150', '161890', '005300', '004490',
       '214320', '091990', '293490', '247540', '068760', '263750', '036490',
       '028300'],
      dtype='object')

In [66]:
test2 = test.copy()

In [67]:
test2.columns

Index(['Date', 'weekday', 'weeknum', 'kosdaq', 'nasdaq', 'sp500', 'dow',
       'kospi', '005930', '000660',
       ...
       '078020', '102710', '314130', '035890', '032620', '220630', '064260',
       '287410', '110790', '099320'],
      dtype='object', length=378)

In [68]:
test2 = test2[['Date', 'weekday', 'weeknum', 'kosdaq', 'nasdaq', 'sp500', 'dow', 'kospi']]
test2.head()

,Date,weekday,weeknum,kosdaq,nasdaq,sp500,dow,kospi
215,2021-11-01,0,44,998.57,15595.9,4613.67,35913.68,2978.94
216,2021-11-02,1,44,1009.44,15649.6,4630.65,36053.09,3013.49
217,2021-11-03,2,44,1005.00,15811.6,4660.57,36157.02,2975.71
218,2021-11-04,3,44,1001.43,15940.3,4680.06,36124.66,2983.22
219,2021-11-05,4,44,1001.35,15971.6,4697.53,36329.07,2969.27


In [69]:
features = list(train.columns)
target_feature = features[8]
basic_features = features[:8] + [target_feature]
basic_features

['Date',
 'weekday',
 'weeknum',
 'kosdaq',
 'nasdaq',
 'sp500',
 'dow',
 'kospi',
 '005930']

In [72]:
stock_pred_dict = {}
features = list(train.columns)
for i in range(8,379):
    # 타겟 특성 선택
    target_feature = features[i]
    # 날짜 3개 + 지수5개 + 타겟 1개 특성 선택
    basic_features = features[:8] + [target_feature]
    
    # 훈련용 데이터 셋 특성 선택
    mod_train = train[basic_features]

    # pycaret으로 훈련, 예측
    model = setup(mod_train, target = target_feature, silent=True)
    tuned_model = tune_model(compare_models(sort='MAPE', n_select=1))
    tuned_model_ = finalize_model(tuned_model)
    # 날짜 3개 + 지수5개로 예측
    pred = predict_model(tuned_model_, test2)
    # 예측값 test셋에 종목에 기록
    test[target_feature] = pred['Label']
    
    # test셋의 종목 종가 dict에 종목 코드로 기록
    stock_pred_dict[target_feature] = test[target_feature]

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,717.3079,1319199.6357,1148.5642,0.9380,0.0137,0.0087
1,1216.7309,2072834.0433,1439.7340,0.9073,0.0185,0.0156
2,1122.3960,2303060.2418,1517.5837,0.8908,0.0193,0.0142
3,662.2780,612751.3416,782.7844,0.9783,0.0100,0.0085
4,749.8314,798650.4674,893.6725,0.9378,0.0109,0.0092
5,811.5114,1936777.4767,1391.6815,0.8485,0.0165,0.0099
6,555.4274,573016.1423,756.9783,0.9749,0.0094,0.0070
7,562.0725,608753.6614,780.2267,0.9629,0.0098,0.0071
8,672.5167,644804.8049,802.9974,0.9751,0.0107,0.0088
9,988.6503,1571863.8436,1253.7399,0.9127,0.0151,0.0118


IndexError: list index out of range

In [64]:
stock_pred_dict

{'005930': 215    74627.246320
 216    73965.025190
 217    71580.217211
 218    71212.728797
 219    73079.304257
 Name: 005930, dtype: float64,
 '000660': 215    109583.694160
 216    108889.693876
 217    102612.827008
 218    104779.673691
 219    109958.428158
 Name: 000660, dtype: float64,
 '035420': 215    408758.653846
 216    409221.153846
 217    406460.096154
 218    409212.500000
 219    409928.365385
 Name: 035420, dtype: float64}

In [65]:
len(stock_pred_dict)

3

In [ ]:
stock_pred = pd.DataFrame(stock_pred_dict)
stock_pred

In [ ]:
stock_pred.columns

In [ ]:
stock_pred * 2

In [ ]:
type(stock_pred['005930'])

In [ ]:
final_stock_pred = pd.concat([stock_pred, stock_pred])
final_stock_pred

In [ ]:
final_stock_pred = final_stock_pred.reset_index()
final_stock_pred

In [ ]:
final_stock_pred = final_stock_pred.drop('index',axis=1)
final_stock_pred

In [ ]:
final_stock_pred.columns

# 제출 파일 만들기

In [ ]:
sample_name = 'sample_submission.csv'
sub = pd.read_csv(os.path.join(path,sample_name))
sub

In [ ]:
for code in final_stock_pred.columns:
    sub[code] = final_stock_pred[code]

In [ ]:
sub

In [ ]:
sub.to_csv('sub01_cat01.csv', index=False)